<a href="https://colab.research.google.com/github/vmartinezarias/Curso_Ecologia_Paisaje_y-Ecoacustica/blob/main/11%20-%20Indices_Acusticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CÁLCULO DE ÍNDICES ACÚSTICOS EMPLEANDO EL PAQUETE "SCIKIT-MAAD"


## Verificar poder de procesamiento

In [ ]:
from psutil import cpu_count
print(f"Número de CPUs disponibles: {cpu_count(logical=True)}")

Número de CPUs disponibles: 96


## Instalación de librerías requeridas

In [ ]:
!pip install scikit-maad
!pip install openpyxl

## Carga de paquetes requeridos y acceso al drive

In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import signal
from maad import sound, features
import torchaudio
from tqdm import tqdm
from google.colab import drive
import warnings
import concurrent.futures

warnings.filterwarnings("ignore")

# Montar Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Definir parámetros de entrada

In [ ]:
# Parámetros de entrada
BASE_PATH = "/content/drive/MyDrive/Curso_Ecologia_Paisaje_Ecoacustica/Audios_SinLluvia"  # Ruta base
OUTPUT_DIR = "/content/drive/MyDrive/Curso_Ecologia_Paisaje_Ecoacustica/Indices_acusticos"  # Directorio de salida
OUTPUT_FILE = f"{OUTPUT_DIR}/IndicesAcusticos0-12_5-15.xlsx"  # Archivo de salida
WORKERS = os.cpu_count() - 2  # Número de procesadores a utilizar

# Crear la carpeta de salida si no existe
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"Directorio creado: {OUTPUT_DIR}")
else:
    print(f"Directorio ya existente: {OUTPUT_DIR}")




Directorio ya existente: /content/drive/MyDrive/Curso_Ecologia_Paisaje_Ecoacustica/Indices_acusticos


## Definir parámetros para índices acústicos


**Window_Size** (Tamaño de ventana):
El tamaño de ventana (como WINDOW_SIZE = 512) determina cuántos puntos de la señal se analizan en cada segmento al calcular un espectrograma con la Transformada Rápida de Fourier (FFT). Es clave porque afecta la resolución temporal y espectral del análisis.

Tamaños pequeños (e.g., 256 puntos): Ofrecen mejor resolución temporal, lo que permite capturar eventos rápidos en el tiempo, pero sacrifican precisión en la frecuencia (bandas más amplias).
Tamaños grandes (e.g., 1024 puntos): Mejoran la resolución espectral, lo que permite identificar frecuencias específicas con mayor precisión, pero sacrifican detalles temporales, dificultando la captura de eventos breves.
Un tamaño de 512 puntos es común, equilibrando la capacidad de capturar eventos en tiempo y frecuencia. En combinación con el solapamiento, permite mantener continuidad entre ventanas consecutivas y obtener un espectrograma más detallado y suave.


**Overlap** (sobrelapamiento): El solapamiento (overlap) es la cantidad de datos compartidos entre ventanas consecutivas de una señal al calcular su espectrograma mediante la Transformada Rápida de Fourier (FFT). Esto implica que, al dividir una señal en ventanas de un tamaño específico (por ejemplo, 512 puntos), una nueva ventana comienza antes de que termine la anterior, permitiendo que se superpongan. Esto ayuda a capturar transiciones rápidas y mejora la continuidad temporal en el análisis. Por ejemplo, con un solapamiento del 50% y una ventana de 1024 puntos, la segunda ventana comenzará en el punto 512 de la primera y continuará superponiéndose.

El solapamiento es útil porque mejora la resolución temporal, reduce la pérdida de información entre ventanas y genera espectrogramas más suaves visualmente.

Solapamiento bajo (< 50%): Puede ser útil si quieres reducir el tiempo de cómputo o si los eventos son lentos.
Solapamiento alto (> 75%): Ideal para señales con cambios rápidos en el tiempo, pero aumenta el costo computacional.
Solapamiento medio (50%): Es una configuración estándar que equilibra bien entre resolución temporal y computación (David Luna, com.pers).


**NFFT** (Puntos FFT):
El número de puntos FFT determina cuántos puntos se calculan en la Transformada Rápida de Fourier (FFT) para cada ventana. Esto afecta directamente la resolución en el dominio de la frecuencia:

Valores pequeños (e.g., 256): Proporcionan menos precisión en las frecuencias (bandas más amplias), pero son más rápidos de calcular.
Valores grandes (e.g., 1024): Ofrecen mayor resolución en frecuencia, dividiendo el espectro en bandas más estrechas, pero requieren más cómputo.
Un valor de NFFT = 512 es común porque equilibra la precisión en la frecuencia con la eficiencia computacional. Normalmente se elige igual al tamaño de la ventana (WINDOW_SIZE) para mantener coherencia en el análisis.

**J_CLUSTER** (El número de clusters para el ACI temporal)
Define cómo se divide la señal en bloques de tiempo al calcular el Índice de Complejidad Acústica (ACI) temporal.

Cada cluster representa un segmento de tiempo sobre el cual se evalúan los cambios en la energía de las frecuencias. Este valor afecta la granularidad del análisis temporal:

Valores bajos (ej., 2-5): Agrupan más datos en cada cluster, lo que suaviza el análisis temporal y detecta tendencias generales.

Valores altos (ej., >10): Aumentan la sensibilidad a cambios rápidos en la señal, capturando eventos más pequeños o transitorios.

In [ ]:
# Parámetros para índices acústicos
DEFAULT_FMIN = 0  # Frecuencia mínima por defecto
DEFAULT_FMAX = 15000  # Frecuencia máxima por defecto, en Hz
BIOPHONY_BAND = (2000, 12000)  # Banda bioacústica para BI y NDSI
ANTHROPHONY_BAND = (0, 2000)  # Banda tecnofónica para NDSI
WINDOW_SIZE = 512  # Tamaño de la ventana para espectrograma
NFFT = 512  # Número de puntos FFT
OVERLAP = 0  # Solapamiento
J_CLUSTER = 5  # Número de clusters para ACI temporal


## Cálculo de los índices acústicos

In [ ]:
# Función para el cálculo de índices
def calculate_H(audio, Fs, s, f, fmin, fmax):
    """Calcula Hf, Ht y H total."""
    # Ht: Entropía temporal
    hilbert_env = np.abs(signal.hilbert(audio[0, :]))
    env_prob = hilbert_env / hilbert_env.sum()
    Ht = -np.sum(env_prob * np.log2(env_prob + 1e-10)) / np.log2(len(env_prob))

    # Hf: Entropía espectral
    freq_limits = np.logical_and(f >= fmin, f <= fmax)
    s_band = s[freq_limits, :].sum(axis=1)
    s_band = s_band / np.sum(s_band + 1e-10)
    Hf = -np.sum(s_band * np.log2(s_band + 1e-10)) / np.log2(len(s_band))

    # H total
    H_total = Ht * Hf
    return Ht, Hf, H_total

def get_indices(file_path, fmin, fmax):
    """Procesa un archivo de audio y calcula los índices acústicos."""
    try:
        audio, fs = torchaudio.load(file_path)
        f, t, s = signal.spectrogram(audio[0, :], fs, nperseg=WINDOW_SIZE, noverlap=OVERLAP, nfft=NFFT)

        # Índices acústicos
        ACItf_val = ACItf(audio[0, :], fs, J_CLUSTER, fmin, fmax, s, f)
        ACIft_val = ACIft(s, f, fmin, fmax)
        BI_val = features.bioacoustics_index(s, f, flim=BIOPHONY_BAND)
        NP_val = number_of_peaks(s, f, fmin, fmax)
        Ht_val, Hf_val, H_val = calculate_H(audio, fs, s, f, fmin, fmax)
        NDSI_val = features.soundscape_index(s, f, flim_bioPh=BIOPHONY_BAND, flim_antroPh=ANTHROPHONY_BAND)[0]

        # Resultado para cada archivo
        return [file_path.name, file_path.parent.name, ACItf_val, ACIft_val, BI_val, NP_val, Ht_val, Hf_val, H_val, NDSI_val]
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return [file_path.name, file_path.parent.name] + [np.nan] * 8

# Procesamiento en paralelo
if __name__ == "__main__":
    path = Path(BASE_PATH)
    files = list(path.rglob("*.[wW][aA][vV]"))
    print(f"Number of files: {len(files)}")

    with concurrent.futures.ProcessPoolExecutor(max_workers=WORKERS) as executor:
        results = list(tqdm(executor.map(get_indices, files, [DEFAULT_FMIN]*len(files), [DEFAULT_FMAX]*len(files)),
                            total=len(files), desc="Processing files"))

    # Crear DataFrame y guardar en Excel
    df = pd.DataFrame(results, columns=["Name", "Folder", "ACItf", "ACIft", "BI", "NP", "Ht", "Hf", "H", "NDSI"])
    df.to_excel(OUTPUT_FILE, index=False)
    print(f"Results saved to: {OUTPUT_FILE}")

# GRÁFICA DE LOS ÍNDICES ACÚSTICOS

## Cargar librerías requeridas y el data rame

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

# Ruta del archivo de entrada en Google Drive
input_file = "/content/drive/MyDrive/Curso_Ecologia_Paisaje_Ecoacustica/Indices_acusticos/IndicesAcusticos0-12.xlsx"

# Ruta para guardar el archivo de salida
output_folder = "/content/drive/MyDrive/Curso_Ecologia_Paisaje_Ecoacustica/Indices_acusticos"
output_file = os.path.join(output_folder, "IndicesPromedioPorHora0-12kHz.xlsx") #cambiar el nombre del archivo de salida!!!

# Crear la carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Leer directamente el archivo Excel en un DataFrame
datos = pd.read_excel(input_file)

## Generar archivo para gráficos

In [ ]:
PuntosMuestreo = datos['Folder'].unique()

# Crear el DataFrame para almacenar los resultados
IndicesPromedioHora = pd.DataFrame(columns=['PuntoMuestreo', 'Hora', 'ACItf', 'ACIft', 'BI', 'NP', 'Ht','Hf','H', 'NDSI'])

# Calcular y guardar los resultados para cada punto de muestreo
for PuntoMuestreo in PuntosMuestreo:
    # Filtrar los datos por punto de muestreo
    subset = datos[datos['Folder'] == PuntoMuestreo]

    # Extraer la hora del nombre del archivo y calcular los valores promedio por hora para cada índice
    indices = ['ACItf', 'ACIft', 'BI', 'NP', 'Ht','Hf','H', 'NDSI']

    for i in range(24):  # Iterar sobre cada hora
        # Filtrar los datos correspondientes a la hora i
        subset_hora = subset[subset['Name'].str.contains(f'_{i:02}')]

        fila = {'PuntoMuestreo': PuntoMuestreo, 'Hora': i}

        for indice in indices:
            if not subset_hora.empty:
                promedio = subset_hora[indice].mean()
            else:
                promedio = np.nan  # Asignar NaN si no hay datos para esa hora
            fila[indice] = promedio

        # Añadir la fila al DataFrame de manera eficiente
        IndicesPromedioHora = pd.concat([IndicesPromedioHora, pd.DataFrame([fila])], ignore_index=True)

# Guardar el DataFrame en el archivo Excel
IndicesPromedioHora.to_excel(output_file, index=False)

print(f"Archivo de resultados guardado en: {output_file}")


## OPCIÓN 1: DESCARGAR EL CÓDIGO (R) Y EJECUTAR EL SHINY EN UN R LOCAL
Se debe tener el xlsx generado en el paso anterior

In [ ]:
#### ESTO ES UN CÓDIGO DE R!!!!!
# REQUIERE INSTALAR PREVIAMENTE LAS LIBRERÍAS QUE SE CARGAN

library(shiny)
library(ggplot2)
library(readxl)

# Definir la UI
ui <- fluidPage(
  titlePanel("Visualización de Índices"),
  sidebarLayout(
    sidebarPanel(
      fileInput("archivo", "Cargar archivo Excel", accept = c(".xlsx")),  # Botón para cargar archivo
      uiOutput("puntos_ui"),  # Menú dinámico para Puntos de Muestreo
      uiOutput("indice_ui")   # Menú dinámico para Índices
    ),
    mainPanel(
      plotOutput("indicePlot")  # Salida del gráfico
    )
  )
)

# Definir el servidor
server <- function(input, output, session) {

  # Leer el archivo cargado
  datos <- reactive({
    req(input$archivo)  # Requiere que se haya cargado un archivo
    read_excel(input$archivo$datapath)  # Leer el archivo cargado
  })

  # Generar el menú dinámico para Puntos de Muestreo
  output$puntos_ui <- renderUI({
    req(datos())  # Requiere que los datos estén cargados
    selectInput("puntoMuestreo", "Puntos de Muestreo:",
                choices = unique(datos()$PuntoMuestreo),
                selected = unique(datos()$PuntoMuestreo)[1],
                multiple = TRUE)
  })

  # Generar el menú dinámico para Índices
  output$indice_ui <- renderUI({
    req(datos())  # Requiere que los datos estén cargados
    selectInput("indice", "Índice:",
                choices = colnames(datos())[3:ncol(datos())],
                selected = colnames(datos())[3])
  })

  # Crear el gráfico
  output$indicePlot <- renderPlot({
    req(input$puntoMuestreo, input$indice)  # Asegura que los inputs estén disponibles
    datos_filtrados <- datos()[datos()$PuntoMuestreo %in% input$puntoMuestreo, ]
    ggplot(datos_filtrados, aes_string(x = 'Hora', y = input$indice, color = 'PuntoMuestreo')) +
      geom_line() +
      labs(x = 'Hora', y = input$indice, title = paste(input$indice, "por Punto de Muestreo")) +
      theme_minimal() +
      theme(legend.title = element_blank())
  })
}

# Ejecutar la aplicación
shinyApp(ui = ui, server = server)

## OPCIÓN 2: PROCÉSELO EN LÍNEA
Esta opción es temporal (hasta 20251231), se avisará en el repositorio cuándo deje de funcionar.

Ingrese a este link: https://vmartinezarias.shinyapps.io/indexvisualization/